In [1]:
import dash
from dash import dcc, html, Input, Output, State, callback, dash_table, ALL
import plotly.graph_objects as go
import numpy as np
import json
import os
import socket
import webbrowser
from datetime import datetime

# Define plot themes for 3D visualization
PLOT_THEMES = {
    'default': {'bg_color': 'white', 'grid_color': '#E5ECF6', 'axis_color': 'black', 'text_color': 'black'},
    'dark': {'bg_color': '#283442', 'grid_color': '#3B4754', 'axis_color': '#EBF0F8', 'text_color': '#EBF0F8'},
    'minimal': {'bg_color': 'white', 'grid_color': '#F5F5F5', 'axis_color': '#666666', 'text_color': '#666666'},
    'night': {'bg_color': '#1a1a1a', 'grid_color': '#333333', 'axis_color': '#999999', 'text_color': '#cccccc'},
    'blueprint': {'bg_color': '#F0F8FF', 'grid_color': '#B0C4DE', 'axis_color': '#4682B4', 'text_color': '#4682B4'}
}

# Initialize Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server

# Utility functions
def create_rotation_matrix(euler_angles, rotation_order):
    """Create rotation matrix from Euler angles and rotation order."""
    angles = np.radians(euler_angles)
    R = np.eye(3)
    for axis, angle in zip(rotation_order.lower(), angles):
        c, s = np.cos(angle), np.sin(angle)
        if axis == 'x':
            R = R @ np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
        elif axis == 'y':
            R = R @ np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
        elif axis == 'z':
            R = R @ np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
    return R

def plot_triad(R, t, color, name):
    """Generate 3D triad traces for Plotly."""
    length = 1.0
    axes = ['X', 'Y', 'Z']
    traces = []
    for i in range(3):
        direction = R[:, i]
        end = t + length * direction
        traces.append(go.Scatter3d(
            x=[t[0], end[0]], y=[t[1], end[1]], z=[t[2], end[2]],
            mode='lines', line=dict(color=color, width=5),
            name=f"{name} {axes[i]}"
        ))
    return traces

# App layout
app.layout = html.Div([
    html.H1("Rigid Load Transfer Tool", style={'textAlign': 'center', 'color': '#2c3e50', 'fontFamily': 'Arial', 'marginBottom': '10px'}),
    
    # Data stores
    dcc.Store(id='parts-store', data=[]),
    dcc.Store(id='connections-store', data=[]),
    dcc.Store(id='targets-store', data=[]),
    dcc.Store(id='gravity-store', data={'direction': [0.0, 0.0, -9.81]}),
    dcc.Download(id="download-data"),
    
    html.Div([
        # Left panel: Inputs
        html.Div([
            # Gravity input
            html.Div([
                html.Label("Gravity (X,Y,Z):", style={'marginRight': '10px'}),
                dcc.Input(id='gravity-x', value=0.0, type='number', style={'width': '50px'}),
                dcc.Input(id='gravity-y', value=0.0, type='number', style={'width': '50px'}),
                dcc.Input(id='gravity-z', value=-9.81, type='number', style={'width': '50px'}),
            ], style={'display': 'flex', 'alignItems': 'center', 'gap': '5px', 'marginBottom': '10px'}),
            
            # Add Part button
            html.Button('➕ Add Part', id='add-part-btn', n_clicks=0, style={
                'width': '100%', 'backgroundColor': '#27ae60', 'color': 'white', 'border': 'none', 'padding': '10px',
                'borderRadius': '5px', 'cursor': 'pointer', 'fontSize': '16px'
            }),
            html.Div(id='parts-inputs-container', style={'marginTop': '10px'}),
            
            html.Hr(style={'border': '1px solid #ccc'}),
            
            # Add Connection button
            html.Button('➕ Add Connection', id='add-connection-btn', n_clicks=0, style={
                'width': '100%', 'backgroundColor': '#e67e22', 'color': 'white', 'border': 'none', 'padding': '10px',
                'borderRadius': '5px', 'cursor': 'pointer', 'fontSize': '16px'
            }),
            html.Div(id='connections-inputs-container', style={'marginTop': '10px'}),
            
            html.Hr(style={'border': '1px solid #ccc'}),
            
            # Add Target Interface button
            html.Button('➕ Add Target Interface', id='add-target-btn', n_clicks=0, style={
                'width': '100%', 'backgroundColor': '#e67e22', 'color': 'white', 'border': 'none', 'padding': '10px',
                'borderRadius': '5px', 'cursor': 'pointer', 'fontSize': '16px'
            }),
            html.Div(id='target-inputs-container', style={'marginTop': '10px'}),
            
            # Export button
            html.Button('💾 Export Data', id='export-btn', style={
                'width': '100%', 'backgroundColor': '#3498db', 'color': 'white', 'border': 'none', 'padding': '10px',
                'borderRadius': '5px', 'cursor': 'pointer', 'fontSize': '16px', 'marginTop': '10px'
            }),
            
            # Theme selector
            html.Div([
                html.Label("Plot Theme:", style={'marginRight': '10px'}),
                dcc.Dropdown(
                    id='theme-selector',
                    options=[{'label': k.capitalize(), 'value': k} for k in PLOT_THEMES.keys()],
                    value='default', style={'width': '200px'}
                )
            ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'flex-end', 'padding': '10px 20px'}),
        ], style={
            'width': '25%', 'padding': '15px', 'borderRadius': '10px', 'backgroundColor': '#ecf0f1',
            'boxShadow': '2px 2px 10px rgba(0,0,0,0.1)', 'height': '80vh', 'overflowY': 'auto'
        }),

        # Right panel: Visualization and Results
        html.Div([
            dcc.Graph(id='3d-plot', style={
                'height': '80%', 'borderRadius': '10px', 'boxShadow': '2px 2px 15px rgba(0,0,0,0.2)', 'backgroundColor': 'white', 'padding': '10px'
            }),
            html.Div(id='results-container', style={'height': '20%', 'marginTop': '10px', 'padding': '10px', 'borderRadius': '10px', 'backgroundColor': '#f9f9f9'}),
            html.Footer('© 2025 Pramod Kumar Yadav (@iAmPramodYadav)'),
        ], style={'width': '75%', 'height': '80vh'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'gap': '20px', 'padding': '20px'})
])

# Callbacks

# Add a new part
@app.callback(
    Output('parts-store', 'data'),
    Input('add-part-btn', 'n_clicks'),
    State('parts-store', 'data'),
    prevent_initial_call=True
)
def add_part(n_clicks, data):
    new_part = {
        'name': f'Part {len(data) + 1}', 'mass': 0.0, 'cog_local': [0.0, 0.0, 0.0],
        'external_force_local': [0.0, 0.0, 0.0], 'external_moment_local': [0.0, 0.0, 0.0],
        'translation': [0.0, 0.0, 0.0], 'euler_angles': [0.0, 0.0, 0.0], 'rotation_order': 'xyz',
        'color': {'hex': f'#{np.random.randint(0, 0xFFFFFF):06x}'}
    }
    return data + [new_part]

# Add a new connection
@app.callback(
    Output('connections-store', 'data'),
    Input('add-connection-btn', 'n_clicks'),
    State('connections-store', 'data'),
    State('parts-store', 'data'),
    prevent_initial_call=True
)
def add_connection(n_clicks, connections, parts):
    if len(parts) < 2:
        return connections
    new_connection = {'parent': parts[0]['name'], 'child': parts[-1]['name']}
    return connections + [new_connection]

# Add a new target interface with coordinate system
@app.callback(
    Output('targets-store', 'data'),
    Input('add-target-btn', 'n_clicks'),
    State('targets-store', 'data'),
    State('connections-store', 'data'),
    prevent_initial_call=True
)
def add_target_interface(n_clicks, targets, connections):
    if not connections:
        return targets
    new_target = {
        'connection': connections[-1],
        'translation': [0.0, 0.0, 0.0],
        'euler_angles': [0.0, 0.0, 0.0],
        'rotation_order': 'xyz'
    }
    return targets + [new_target]

# Update parts inputs UI
@app.callback(
    Output('parts-inputs-container', 'children'),
    Input('parts-store', 'data')
)
def update_parts_inputs(parts):
    controls = []
    for i, part in enumerate(parts):
        system_color = part['color']['hex']
        controls.append(html.Div([
            html.H5(f"Part {i+1}"),
            html.Div([html.Label("Name:"), dcc.Input(value=part['name'], type='text', id={'type': 'part-name', 'index': i}, style={'width': '100px'})]),
            html.Div([html.Label("Mass:"), dcc.Input(value=part['mass'], type='number', id={'type': 'part-mass', 'index': i}, style={'width': '50px'})]),
            html.Div([html.Label("COG Local (X,Y,Z):"),
                      dcc.Input(value=part['cog_local'][0], type='number', id={'type': 'part-cog-local-x', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['cog_local'][1], type='number', id={'type': 'part-cog-local-y', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['cog_local'][2], type='number', id={'type': 'part-cog-local-z', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
            html.Div([html.Label("External Force Local (X,Y,Z):"),
                      dcc.Input(value=part['external_force_local'][0], type='number', id={'type': 'part-fx-local', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['external_force_local'][1], type='number', id={'type': 'part-fy-local', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['external_force_local'][2], type='number', id={'type': 'part-fz-local', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
            html.Div([html.Label("External Moment Local (X,Y,Z):"),
                      dcc.Input(value=part['external_moment_local'][0], type='number', id={'type': 'part-mx-local', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['external_moment_local'][1], type='number', id={'type': 'part-my-local', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['external_moment_local'][2], type='number', id={'type': 'part-mz-local', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
            html.Div([html.Label("Translation (X,Y,Z):"),
                      dcc.Input(value=part['translation'][0], type='number', id={'type': 'part-tx', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['translation'][1], type='number', id={'type': 'part-ty', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['translation'][2], type='number', id={'type': 'part-tz', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
            html.Div([html.Label("Rotation Order:"), dcc.Dropdown(options=['xyz', 'xzy', 'yxz', 'yzx', 'zxy', 'zyx'], value=part['rotation_order'], id={'type': 'part-rot-order', 'index': i}, style={'width': '100px'})]),
            html.Div([html.Label("Euler Angles (deg):"),
                      dcc.Input(value=part['euler_angles'][0], type='number', id={'type': 'part-rx', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['euler_angles'][1], type='number', id={'type': 'part-ry', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=part['euler_angles'][2], type='number', id={'type': 'part-rz', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
        ], style={'border': f'2px solid {system_color}', 'padding': '10px', 'marginBottom': '10px'}))
    return controls

# Update connections inputs UI
@app.callback(
    Output('connections-inputs-container', 'children'),
    Input('connections-store', 'data'),
    State('parts-store', 'data')
)
def update_connections_inputs(connections, parts):
    part_names = [part['name'] for part in parts]
    controls = []
    for i, conn in enumerate(connections):
        controls.append(html.Div([
            html.Label(f"Connection {i+1}:"),
            dcc.Dropdown(options=part_names, value=conn['parent'], id={'type': 'conn-parent', 'index': i}, style={'width': '100px'}),
            html.Label("to"),
            dcc.Dropdown(options=part_names, value=conn['child'], id={'type': 'conn-child', 'index': i}, style={'width': '100px'}),
        ], style={'display': 'flex', 'gap': '5px', 'marginBottom': '5px'}))
    return controls

# Update target inputs UI with coordinate system
@app.callback(
    Output('target-inputs-container', 'children'),
    Input('targets-store', 'data'),
    State('connections-store', 'data')
)
def update_targets_inputs(targets, connections):
    conn_options = [{'label': f"{conn['parent']} to {conn['child']}", 'value': json.dumps(conn)} for conn in connections]
    controls = []
    for i, target in enumerate(targets):
        controls.append(html.Div([
            html.H5(f"Target {i+1}"),
            html.Div([html.Label("Connection:"), dcc.Dropdown(options=conn_options, value=json.dumps(target['connection']), id={'type': 'target-conn', 'index': i}, style={'width': '200px'})]),
            html.Div([html.Label("Translation (X,Y,Z):"),
                      dcc.Input(value=target['translation'][0], type='number', id={'type': 'target-tx', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=target['translation'][1], type='number', id={'type': 'target-ty', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=target['translation'][2], type='number', id={'type': 'target-tz', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
            html.Div([html.Label("Rotation Order:"), dcc.Dropdown(options=['xyz', 'xzy', 'yxz', 'yzx', 'zxy', 'zyx'], value=target['rotation_order'], id={'type': 'target-rot-order', 'index': i}, style={'width': '100px'})]),
            html.Div([html.Label("Euler Angles (deg):"),
                      dcc.Input(value=target['euler_angles'][0], type='number', id={'type': 'target-rx', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=target['euler_angles'][1], type='number', id={'type': 'target-ry', 'index': i}, style={'width': '50px'}),
                      dcc.Input(value=target['euler_angles'][2], type='number', id={'type': 'target-rz', 'index': i}, style={'width': '50px'})],
                     style={'display': 'flex', 'gap': '5px'}),
        ], style={'border': '1px solid #ccc', 'padding': '10px', 'marginBottom': '10px'}))
    return controls

# Update all stores based on user inputs
@app.callback(
    [Output('parts-store', 'data', allow_duplicate=True),
     Output('connections-store', 'data', allow_duplicate=True),
     Output('targets-store', 'data', allow_duplicate=True),
     Output('gravity-store', 'data', allow_duplicate=True)],
    [Input({'type': 'part-name', 'index': ALL}, 'value'),
     Input({'type': 'part-mass', 'index': ALL}, 'value'),
     Input({'type': 'part-cog-local-x', 'index': ALL}, 'value'),
     Input({'type': 'part-cog-local-y', 'index': ALL}, 'value'),
     Input({'type': 'part-cog-local-z', 'index': ALL}, 'value'),
     Input({'type': 'part-fx-local', 'index': ALL}, 'value'),
     Input({'type': 'part-fy-local', 'index': ALL}, 'value'),
     Input({'type': 'part-fz-local', 'index': ALL}, 'value'),
     Input({'type': 'part-mx-local', 'index': ALL}, 'value'),
     Input({'type': 'part-my-local', 'index': ALL}, 'value'),
     Input({'type': 'part-mz-local', 'index': ALL}, 'value'),
     Input({'type': 'part-tx', 'index': ALL}, 'value'),
     Input({'type': 'part-ty', 'index': ALL}, 'value'),
     Input({'type': 'part-tz', 'index': ALL}, 'value'),
     Input({'type': 'part-rot-order', 'index': ALL}, 'value'),
     Input({'type': 'part-rx', 'index': ALL}, 'value'),
     Input({'type': 'part-ry', 'index': ALL}, 'value'),
     Input({'type': 'part-rz', 'index': ALL}, 'value'),
     Input({'type': 'conn-parent', 'index': ALL}, 'value'),
     Input({'type': 'conn-child', 'index': ALL}, 'value'),
     Input({'type': 'target-conn', 'index': ALL}, 'value'),
     Input({'type': 'target-tx', 'index': ALL}, 'value'),
     Input({'type': 'target-ty', 'index': ALL}, 'value'),
     Input({'type': 'target-tz', 'index': ALL}, 'value'),
     Input({'type': 'target-rot-order', 'index': ALL}, 'value'),
     Input({'type': 'target-rx', 'index': ALL}, 'value'),
     Input({'type': 'target-ry', 'index': ALL}, 'value'),
     Input({'type': 'target-rz', 'index': ALL}, 'value'),
     Input('gravity-x', 'value'),
     Input('gravity-y', 'value'),
     Input('gravity-z', 'value')],
    [State('parts-store', 'data'),
     State('connections-store', 'data'),
     State('targets-store', 'data'),
     State('gravity-store', 'data')],
    prevent_initial_call=True
)
def update_stores(*args):
    parts, connections, targets, gravity = args[-4:]
    ctx = dash.callback_context
    if not ctx.triggered:
        return parts, connections, targets, gravity

    # Update parts
    for i in range(len(parts)):
        parts[i]['name'] = ctx.inputs_list[0][i]['value'] or parts[i]['name']
        parts[i]['mass'] = ctx.inputs_list[1][i]['value'] if ctx.inputs_list[1][i]['value'] is not None else parts[i]['mass']
        parts[i]['cog_local'] = [
            ctx.inputs_list[2][i]['value'] if ctx.inputs_list[2][i]['value'] is not None else parts[i]['cog_local'][0],
            ctx.inputs_list[3][i]['value'] if ctx.inputs_list[3][i]['value'] is not None else parts[i]['cog_local'][1],
            ctx.inputs_list[4][i]['value'] if ctx.inputs_list[4][i]['value'] is not None else parts[i]['cog_local'][2]
        ]
        parts[i]['external_force_local'] = [
            ctx.inputs_list[5][i]['value'] if ctx.inputs_list[5][i]['value'] is not None else parts[i]['external_force_local'][0],
            ctx.inputs_list[6][i]['value'] if ctx.inputs_list[6][i]['value'] is not None else parts[i]['external_force_local'][1],
            ctx.inputs_list[7][i]['value'] if ctx.inputs_list[7][i]['value'] is not None else parts[i]['external_force_local'][2]
        ]
        parts[i]['external_moment_local'] = [
            ctx.inputs_list[8][i]['value'] if ctx.inputs_list[8][i]['value'] is not None else parts[i]['external_moment_local'][0],
            ctx.inputs_list[9][i]['value'] if ctx.inputs_list[9][i]['value'] is not None else parts[i]['external_moment_local'][1],
            ctx.inputs_list[10][i]['value'] if ctx.inputs_list[10][i]['value'] is not None else parts[i]['external_moment_local'][2]
        ]
        parts[i]['translation'] = [
            ctx.inputs_list[11][i]['value'] if ctx.inputs_list[11][i]['value'] is not None else parts[i]['translation'][0],
            ctx.inputs_list[12][i]['value'] if ctx.inputs_list[12][i]['value'] is not None else parts[i]['translation'][1],
            ctx.inputs_list[13][i]['value'] if ctx.inputs_list[13][i]['value'] is not None else parts[i]['translation'][2]
        ]
        parts[i]['rotation_order'] = ctx.inputs_list[14][i]['value'] or parts[i]['rotation_order']
        parts[i]['euler_angles'] = [
            ctx.inputs_list[15][i]['value'] if ctx.inputs_list[15][i]['value'] is not None else parts[i]['euler_angles'][0],
            ctx.inputs_list[16][i]['value'] if ctx.inputs_list[16][i]['value'] is not None else parts[i]['euler_angles'][1],
            ctx.inputs_list[17][i]['value'] if ctx.inputs_list[17][i]['value'] is not None else parts[i]['euler_angles'][2]
        ]

    # Update connections
    for i in range(len(connections)):
        connections[i]['parent'] = ctx.inputs_list[18][i]['value'] or connections[i]['parent']
        connections[i]['child'] = ctx.inputs_list[19][i]['value'] or connections[i]['child']

    # Update targets
    for i in range(len(targets)):
        if i < len(ctx.inputs_list[20]):
            targets[i]['connection'] = json.loads(ctx.inputs_list[20][i]['value']) if ctx.inputs_list[20][i]['value'] else targets[i]['connection']
        if i < len(ctx.inputs_list[21]):
            targets[i]['translation'][0] = ctx.inputs_list[21][i]['value'] if ctx.inputs_list[21][i]['value'] is not None else targets[i]['translation'][0]
        if i < len(ctx.inputs_list[22]):
            targets[i]['translation'][1] = ctx.inputs_list[22][i]['value'] if ctx.inputs_list[22][i]['value'] is not None else targets[i]['translation'][1]
        if i < len(ctx.inputs_list[23]):
            targets[i]['translation'][2] = ctx.inputs_list[23][i]['value'] if ctx.inputs_list[23][i]['value'] is not None else targets[i]['translation'][2]
        if i < len(ctx.inputs_list[24]):
            targets[i]['rotation_order'] = ctx.inputs_list[24][i]['value'] or targets[i]['rotation_order']
        if i < len(ctx.inputs_list[25]):
            targets[i]['euler_angles'][0] = ctx.inputs_list[25][i]['value'] if ctx.inputs_list[25][i]['value'] is not None else targets[i]['euler_angles'][0]
        if i < len(ctx.inputs_list[26]):
            targets[i]['euler_angles'][1] = ctx.inputs_list[26][i]['value'] if ctx.inputs_list[26][i]['value'] is not None else targets[i]['euler_angles'][1]
        if i < len(ctx.inputs_list[27]):
            targets[i]['euler_angles'][2] = ctx.inputs_list[27][i]['value'] if ctx.inputs_list[27][i]['value'] is not None else targets[i]['euler_angles'][2]

    # Update gravity
    gravity['direction'] = [
        ctx.inputs_list[28]['value'] if ctx.inputs_list[28]['value'] is not None else gravity['direction'][0],
        ctx.inputs_list[29]['value'] if ctx.inputs_list[29]['value'] is not None else gravity['direction'][1],
        ctx.inputs_list[30]['value'] if ctx.inputs_list[30]['value'] is not None else gravity['direction'][2]
    ]
    return parts, connections, targets, gravity

# Update visualization and results
@app.callback(
    [Output('3d-plot', 'figure'),
     Output('results-container', 'children')],
    [Input('parts-store', 'data'),
     Input('connections-store', 'data'),
     Input('targets-store', 'data'),
     Input('gravity-store', 'data'),
     Input('theme-selector', 'value')]
)
def update_visualization(parts, connections, targets, gravity, theme):
    fig = go.Figure()
    theme_colors = PLOT_THEMES[theme]
    gravity_vector = np.array(gravity['direction'])

    # Build tree structure
    tree = {part['name']: {'data': part, 'children': [], 'parent': None} for part in parts}
    for conn in connections:
        if conn['parent'] != conn['child']:  # Prevent self-loops
            tree[conn['parent']]['children'].append(conn['child'])
            tree[conn['child']]['parent'] = conn['parent']

    # Find root (part with no parent)
    root_name = next((name for name, part in tree.items() if part['parent'] is None), None)
    if not root_name:
        return fig, "No root part found."

    # Compute global transformations and plot triads for parts
    def compute_global_transform(part_name, R_cumulative, t_cumulative):
        part = tree[part_name]['data']
        R_local = create_rotation_matrix(part['euler_angles'], part['rotation_order'])
        t_local = np.array(part['translation'])
        R_global = R_cumulative @ R_local
        t_global = t_cumulative + R_cumulative @ t_local
        tree[part_name]['R_global'] = R_global
        tree[part_name]['t_global'] = t_global
        for trace in plot_triad(R_global, t_global, part['color']['hex'], part['name']):
            fig.add_trace(trace)
        for child_name in tree[part_name]['children']:
            compute_global_transform(child_name, R_global, t_global)
        return R_global, t_global

    compute_global_transform(root_name, np.eye(3), np.array([0.0, 0.0, 0.0]))

    # Recursive load calculation for subtree in global coordinates
    def calculate_subtree_loads(part_name):
        part = tree[part_name]['data']
        R_global = tree[part_name]['R_global']
        t_global = tree[part_name]['t_global']
        cog_global = t_global + R_global @ part['cog_local'] if part['mass'] > 0 else t_global
        F_weight_global = part['mass'] * gravity_vector if part['mass'] > 0 else np.zeros(3)
        F_external_global = R_global @ part['external_force_local']
        M_external_global = R_global @ part['external_moment_local']
        F_own_global = F_weight_global + F_external_global
        M_own_global = np.cross(cog_global, F_weight_global) + np.cross(t_global, F_external_global) + M_external_global

        F_subtree_global = F_own_global.copy()
        M_subtree_global = M_own_global.copy()
        for child_name in tree[part_name]['children']:
            F_child, M_child = calculate_subtree_loads(child_name)
            F_subtree_global += F_child
            M_subtree_global += M_child
        return F_subtree_global, M_subtree_global

    # Plot targets and calculate loads
    results = []
    for i, target in enumerate(targets):
        conn = target['connection']
        child_name = conn['child']
        F_subtree_global, M_subtree_global = calculate_subtree_loads(child_name)
        R_target = create_rotation_matrix(target['euler_angles'], target['rotation_order'])
        t_target = np.array(target['translation'])
        for trace in plot_triad(R_target, t_target, 'gray', f"Target {i+1}"):
            fig.add_trace(trace)

        # Transfer loads to target coordinate system
        F_target = R_target.T @ F_subtree_global
        M_target = R_target.T @ (M_subtree_global - np.cross(t_target, F_subtree_global))
        results.append({
            'System': f"Target {i+1}: {conn['parent']} to {conn['child']}",
            'Fx': f"{F_target[0]:.2f}", 'Fy': f"{F_target[1]:.2f}", 'Fz': f"{F_target[2]:.2f}",
            'Mx': f"{M_target[0]:.2f}", 'My': f"{M_target[1]:.2f}", 'Mz': f"{M_target[2]:.2f}"
        })

    # Update plot layout
    fig.update_layout(
        paper_bgcolor=theme_colors['bg_color'], plot_bgcolor=theme_colors['bg_color'],
        scene=dict(
            xaxis=dict(title='X', backgroundcolor=theme_colors['bg_color'], gridcolor=theme_colors['grid_color'], color=theme_colors['text_color']),
            yaxis=dict(title='Y', backgroundcolor=theme_colors['bg_color'], gridcolor=theme_colors['grid_color'], color=theme_colors['text_color']),
            zaxis=dict(title='Z', backgroundcolor=theme_colors['bg_color'], gridcolor=theme_colors['grid_color'], color=theme_colors['text_color']),
            aspectmode='cube', camera=dict(up=dict(x=0, y=0, z=1))
        ),
        margin=dict(l=0, r=0, b=0, t=30), showlegend=True, font=dict(color=theme_colors['text_color'])
    )

    # Create results table
    table = dash_table.DataTable(
        columns=[{'name': col, 'id': col} for col in ['System', 'Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']],
        data=results,
        style_cell={'textAlign': 'center', 'backgroundColor': theme_colors['bg_color'], 'color': theme_colors['text_color']},
        style_header={'backgroundColor': theme_colors['grid_color'], 'fontWeight': 'bold', 'color': theme_colors['text_color']}
    )
    return fig, table

# Export data
@app.callback(
    Output('download-data', 'data'),
    Input('export-btn', 'n_clicks'),
    [State('parts-store', 'data'),
     State('connections-store', 'data'),
     State('targets-store', 'data'),
     State('gravity-store', 'data'),
     State('results-container', 'children')],
    prevent_initial_call=True
)
def export_data(n_clicks, parts, connections, targets, gravity, results):
    content = "=== Rigid Load Transfer Analysis Report ===\n\n"
    content += f"Gravity: {gravity['direction']}\n\n"
    
    content += "=== Parts ===\n"
    for part in parts:
        content += f"{part['name']}:\n  Mass: {part['mass']}\n  COG Local: {part['cog_local']}\n  External Force Local: {part['external_force_local']}\n  External Moment Local: {part['external_moment_local']}\n  Translation: {part['translation']}\n  Euler Angles: {part['euler_angles']}\n  Rotation Order: {part['rotation_order']}\n\n"
    
    content += "=== Connections ===\n"
    for conn in connections:
        content += f"{conn['parent']} to {conn['child']}\n"
    
    content += "=== Targets ===\n"
    for i, target in enumerate(targets):
        content += f"Target {i+1}:\n  Connection: {target['connection']['parent']} to {target['connection']['child']}\n  Translation: {target['translation']}\n  Euler Angles: {target['euler_angles']}\n  Rotation Order: {target['rotation_order']}\n\n"
    
    content += "=== Results ===\n"
    if results and 'props' in results and 'data' in results['props']:
        for row in results['props']['data']:
            content += f"{row['System']}:\n  Force: {row['Fx']}, {row['Fy']}, {row['Fz']}\n  Moment: {row['Mx']}, {row['My']}, {row['Mz']}\n\n"
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"RLT_Report_{timestamp}.txt"
    return dict(content=content, filename=filename)

# # Find a free port
# def find_free_port():
#     sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     sock.bind(("", 0))
#     port = sock.getsockname()[1]
#     sock.close()
#     return port

# Run the app
if __name__ == '__main__':
    # port = int(os.environ.get("PORT", find_free_port()))
    # if os.environ.get("PORT") is None:
    #     url = f"http://localhost:{port}"
    #     webbrowser.open_new(url)
    # app.run_server(host="0.0.0.0", port=port, debug=False)
    app.run_server(debug=True)